In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from src.data_analysis.search import search, path, SearchConfig, l2_distance
from src.data_analysis.height_data import get_height_data_around_point, meter_in_arcseconds
import math

In [ ]:
grid = get_height_data_around_point(47.31701622137232, 11.278839111328125)

grid = grid.downsample(0.5)

In [ ]:
plt.imshow(grid.heights[::-1], interpolation='none')
plt.show()

grid.heights.min(), grid.heights.max()

In [ ]:
start_ix = (grid.heights.shape[0]//2, grid.heights.shape[0]//2)
start_height = grid.heights[start_ix] + 10

state = search(start_ix, start_height, SearchConfig(grid, 1/10))

In [ ]:
heights = np.zeros_like(grid.heights)
heights[:] = np.nan

for (a,b), node in state.explored.items():
    if node.reachable:
        heights[a,b] = node.height
    else:
        heights[a,b] = np.nan

In [ ]:
plt.imshow(heights[::-1], interpolation='none')
plt.show()

In [ ]:
h = heights.copy()

p = path((45, 35), state.explored)

for a, b in zip(p, p[1:]):
    length = l2_distance(a.ix, b.ix)

    i_len = math.ceil(length)
    x, y = np.linspace(a.ix[0], b.ix[0], i_len), np.linspace(a.ix[1], b.ix[1], i_len)

    h[x.astype(int), y.astype(int)] = heights.max()+3
    #h[x.astype(int) + 1, y.astype(int)] = heights.max()+3
    #h[x.astype(int) - 1, y.astype(int)] = heights.max()+3
    #h[x.astype(int), y.astype(int) + 1] = heights.max()+3
    #h[x.astype(int), y.astype(int) - 1] = heights.max()+3

plt.imshow(h, interpolation='none')
plt.show()

In [ ]:
agl = heights - grid.heights

In [ ]:
def crop_to_not_nan(arr: np.array):
    not_na = ~np.isnan(arr)

    rows = np.any(not_na, axis=0)
    cols = np.any(not_na, axis=1)

    r_min = np.min(np.arange(arr.shape[1])[rows])
    r_max = np.max(np.arange(arr.shape[1])[rows])

    c_min = np.min(np.arange(arr.shape[0])[cols])
    c_max = np.max(np.arange(arr.shape[0])[cols])

    return arr[c_min:c_max+1, r_min:r_max+1]

In [ ]:
crop_to_not_nan(agl[5:])

In [ ]:
X, Y = np.indices(grid.heights.T.shape)
fig, ax = plt.subplots(figsize=(6,6))
ax.contour(X,Y,agl.T, levels=50)
plt.show()

In [ ]:
plt.imshow(agl[::-1], interpolation='none')
plt.show()

In [ ]:
x = state.intersection_checks.copy()

print(state.intersection_checks.sum(), state.intersection_checks.mean(), (state.intersection_checks>0).sum(), (state.intersection_checks>0).mean())

plt.imshow(x > 0, interpolation='none')
plt.show()

In [ ]:
plt.imshow(x, interpolation='none')
plt.show()

In [ ]:
steps = np.array([1,0,0,1,1,0,0,0,1]).reshape((3,3))
fractions = np.array([0,0.3,1.0])

m = np.array([0.1, 0.4, 0.6, 0.9]).reshape((2,2))

In [ ]:
c_m = len(m.shape)
expand_to_m = [None]*c_m

f = (m[None] - fractions[:-1][:, *expand_to_m])/(fractions[1:][:, *expand_to_m]-fractions[:-1][:, *expand_to_m])

step_diffs = steps[1:] - steps[:-1]
step_diffs.shape, f.shape

In [ ]:
result = steps[:-1].T[:, :, *expand_to_m] + step_diffs.T[:, :, *expand_to_m]*f

In [ ]:
result.shape, f.shape

result[:, f <= 0] = 0
result[:, f > 1] = 0
result = result.sum(axis=1)

In [ ]:
result